In [62]:

import sys
from os import environ
sys.path.append("./config/utils")

from langchain_openai import OpenAIEmbeddings
from langchain_milvus.vectorstores import Milvus

from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain



from MilvusControll import embeddings
from MilvusControll import OPENAI_API_KEY
from Terminal import terminal_write

In [63]:
vectorstore=Milvus(embeddings,collection_name='docs')

In [64]:
retriever=vectorstore.as_retriever(search_kwargs=dict(k=5))

In [65]:
ids=[document.metadata['id'] for document in retriever.invoke('what 132')]

In [66]:
ids

['f33ae36f-f785-4ccc-96d3-47c534eb1d93',
 'a29a2ed9-49d3-4dce-9ece-d17b9d052ceb',
 '979d539e-8c4f-476f-b2d9-86e2fd8f2920',
 '4bac359e-9fe0-4da5-9632-205ee5a5771d',
 '382a4fa1-846a-4cfe-bcd8-48c231cf8215']

In [22]:
# USPERT
# vectorstore.upsert

#DELETE
# vector_db.delete

In [23]:
ids=[document.metadata['id'] for document in retriever.invoke('what 132')]

In [24]:
vectorstore.delete(ids=ids)

(insert count: 0, delete count: 0, upsert count: 0, timestamp: 0, success count: 0, err count: 0, cost: 0)

In [25]:
documents=retriever.invoke('what 132')

In [26]:
ids=[document.metadata['id'] for document in documents]
ids

['f33ae36f-f785-4ccc-96d3-47c534eb1d93',
 'a29a2ed9-49d3-4dce-9ece-d17b9d052ceb',
 '979d539e-8c4f-476f-b2d9-86e2fd8f2920',
 '4bac359e-9fe0-4da5-9632-205ee5a5771d',
 '382a4fa1-846a-4cfe-bcd8-48c231cf8215']

In [27]:
documents

[Document(metadata={'id': 'f33ae36f-f785-4ccc-96d3-47c534eb1d93', 'source': 'docs/school_informations.txt'}, page_content='School 130|||Armenia, Yerevan, Hakob Hakobyan St. Lane, 3 Building (Arabkir adm. district)\nSchool 131|||Armenia, Yerevan, Arzumanyan St., 22 Building (Ajapnyak adm. district)\nSchool 132|||Armenia, Yerevan, Aram Khachatryan St., 28 Building (Arabkir adm. district)\nSchool 133|||Armenia, Yerevan, Nor Nork 2nd Block, Moldovakan St., 35 Building (Nor Nork adm. district)\nSchool 134|||Armenia, Yerevan, Nor Nork 1st Block, Nanseni St., 14 Building (Nor Nork adm. district)'),
 Document(metadata={'id': 'a29a2ed9-49d3-4dce-9ece-d17b9d052ceb', 'source': 'docs/school_informations.txt'}, page_content='School 134|||Armenia, Yerevan, Nor Nork 1st Block, Nanseni St., 14 Building (Nor Nork adm. district)\nSchool 135|||Armenia, Yerevan, Nor Nork 2nd Block, Nelson Stepanyan St., 5 Building (Nor Nork adm. district)\nSchool 136|||Armenia, Yerevan, Lepsiusi St., 6 Building (Kanaker-Z

In [67]:
llm=ChatOpenAI(api_key=OPENAI_API_KEY, model = 'gpt-3.5-turbo')
retriever=vectorstore.as_retriever(search_kwargs=dict(k=5))
retrieval_qa=RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=False)

In [70]:
queries=['what 132 school', 'what 76 ']

In [71]:
retrieval_qa({'query':queries[0]})

{'query': 'what 132 school',
 'result': 'School 132 is located at Armenia, Yerevan, Aram Khachatryan St., 28 Building in the Arabkir administrative district.'}

In [73]:
conversation_retrieval=ConversationalRetrievalChain.from_llm(
                                    llm=ChatOpenAI(temperature=0.8, model = 'gpt-3.5-turbo'),
                                    retriever=retriever,
                                    return_source_documents=True,
                                                            )

In [74]:
chat_history=[]
while True:
    question=input("question ---> ")
    if question =='OFF':
        break
    
    response = conversation_retrieval(dict(
                            question=question, 
                            chat_history=chat_history)
                                )
    answer=response['answer']
    terminal_write(f"answer ---> {answer}")
    chat_history.append( (question, answer) )
    

question ---> what 132
answer ---> School 132 is located in Armenia, Yerevan, Aram Khachatryan St., building 28 in the Arabkir administrative district.
question ---> на русском
answer ---> Школа 132 в Ереване расположена по адресу: Армения, Ереван, улица Арама Хачатряна, дом 28, в административном районе Арабкир.
question ---> OFF
